# Machine Learning-Used Car Price Predictor-Simple

In [181]:
import warnings
warnings.filterwarnings('ignore')

In [182]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,inspect, func
from config import password
import matplotlib.pyplot as plt
import seaborn as sn
import sqlite3

In [183]:
con = sqlite3.connect("unnormcleanedcar.sqlite")
df = pd.read_sql_query("SELECT * from unnormalizedcarsdb", con)

In [184]:
df = df[['Price', 'Milage', 'Year', 'Model', 'Car Type']]

In [185]:
df.describe()

,Price,Milage,Year
count,23677.000000,23677.000000,23677.000000
mean,29529.070026,83959.113359,2016.588588
std,17251.225185,63801.946178,3.075055
min,1495.000000,10.000000,1995.000000
25%,16786.000000,37344.000000,2015.000000
50%,25985.000000,69000.000000,2017.000000
75%,38987.000000,116000.000000,2019.000000
max,169800.000000,500000.000000,2021.000000


In [186]:
df.shape

(23677, 5)

In [187]:
#label encoding
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

le = preprocessing.LabelEncoder()

for i in range(0,df.shape[1]):
    if df.dtypes[i]=='object':
        df[df.columns[i]] = le.fit_transform(df[df.columns[i]])

In [188]:
df.head()

,Price,Milage,Year,Model,Car Type
0,33888,64965,2018,217,13
1,46980,100000,2018,217,13
2,29995,88300,2016,217,13
3,43701,74900,2020,217,13
4,49950,73987,2019,217,13


In [205]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


scaleddf=scaler.fit_transform(df.values)

In [206]:
#splitting train and test data
from sklearn.model_selection import train_test_split

               
X = scaleddf[['Car Type','Model', 'Milage', 'Year']]
y= scaleddf['Price']
                
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [204]:
X

,Car Type,Model,Milage,Year
0,13,217,64965,2018
1,13,217,100000,2018
2,13,217,88300,2016
3,13,217,74900,2020
4,13,217,73987,2019
5,13,217,58443,2018
6,13,217,74526,2016
7,13,217,35400,2016
8,13,217,79950,2019
9,13,217,133000,2016


In [200]:
#fitting the data in a linear regression model
from sklearn.linear_model import LinearRegression
from sklearn import metrics

lm = LinearRegression()
lm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [201]:
lm.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [202]:
y_pred = lm.predict(X_test)

In [203]:
from sklearn.metrics import r2_score
r2_score(y_pred, y_test, multioutput='variance_weighted')

-1.45154203303382

In [155]:
lmdf = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
lmdf

,Actual,Predicted
14050,9300,14439.232288
16542,28993,40758.172850
21208,46163,42118.378288
1698,38895,36610.691985
2683,39999,30420.109248
7032,28000,36556.436733
12322,39492,35189.989984
16153,9500,19079.324313
12850,17500,20703.255227
16006,32888,36466.619886
